In [ ]:
import clr
import sys, os

import struct
pypath = r'dir\bonsai\Packages'

def add_folder(path):
    for dir_name in os.listdir(path):
        dir_path = os.path.join(path, dir_name)
        if os.path.isdir(dir_path):
            sys.path.insert(0, dir_path)
            add_folder(dir_path)

add_folder(pypath)

clr.AddReference("Bonsai.Harp")

from System import Byte, Array
import System
import Bonsai.Harp as harp

In [ ]:
a = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))

a = harp.HarpMessage.FromSingle(32, harp.MessageType.Write, 1.0)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))
print(harp.HarpMessage.GetPayloadSingle(a))


In [ ]:
a = harp.HarpMessage.FromByte(32, harp.MessageType.Write, 20)
print(harp.HarpMessage.get_Address(a))
print(harp.HarpMessage.get_PayloadType(a))
print(harp.HarpMessage.GetPayloadByte(a))

a.Address

In [ ]:
device = harp.AsyncDevice("COM5")


In [ ]:
device = harp.AsyncDevice
device.ReadHardwareVersionAsync.wait